

Давайте создадим полную и ясную картину всех данных, которые мы используем, откуда они берутся и для чего нужны. Это поможет вам видеть всю систему целиком.

Мы разделим все данные на четыре основные группы.

---

### 1. Данные продукта (Изображение) — "Что мы обрабатываем?"

Это ваши основные данные — само изображение SAR.

| Данные | Источник | Что это такое | Как мы это используем |
| :--- | :--- | :--- | :--- |
| **`.pkl` файлы** (например, `rangeLine_3214_swath_10.pkl`) | Результат вашей предыдущей работы по декодированию сырых данных. | Каждый файл содержит **один целый burst** (вспышку) — двумерный массив комплексных чисел (строки дальности x строки азимута). | **Это входные данные для нашего алгоритма.** Мы будем применять коррекцию к каждому такому массиву. |
| **Имя файла** (например, `swath_10`, номер `3214`) | Имя файла, которое вы сгенерировали при декодировании. | `swath_10` -> это под-полоса **IW1**.<br>`swath_11` -> это **IW2**.<br>`swath_12` -> это **IW3**.<br>Номер `3214` — номер первой строки дальности в этом burst'е. | **Для фильтрации.** Мы используем номер строки, чтобы рассчитать время burst'а и понять, относится ли он к нашему `slice #15`. |

---

### 2. Основные метаданные (Карта продукта) — "Где и когда это было снято?"

Это XML-файлы из архива `S1A_IW_SLC__1SDV_...`. Они описывают ваши данные.

| Данные | Источник (файл и путь) | Что это такое | Как мы это используем |
| :--- | :--- | :--- | :--- |
| **`<ascendingNodeTime>`** | `manifest.safe` | Точное время, когда спутник пересек экватор. | **Это основа времени.** Используется для расчета `Δ_t` для каждого burst'а. |
| **`<sliceList>`** | `annotation/...xml` -> `<imageInformation>` | Список всех "кусков" (slices) полной сцены с их *плановыми* временными границами. | **Чтобы найти временные границы нашего slice #15.** Мы ищем в этом списке `<slice>` с `<sliceNumber>15</sliceNumber>` и берем его `<sensingStartTime>` и `<sensingStopTime>`. |
| **`<productFirstLineUtcTime>`** | `annotation/...xml` -> `<imageInformation>` | Фактическое время первой строки в *этом файле*. | **Вторичный источник для проверки границ slice.** `sliceList` считается более надежным для этой задачи. |
| **`<swathTiming>`** | `annotation/...xml` (несколько секций) | Информация о каждой под-полосе (IW1, IW2, IW3). | **Ключевой источник для расчета времени burst'ов.** |
| **`<burstList>`** | `annotation/...xml` -> `<swathTiming>` | Список всех burst'ов в данной под-полосе с их временными метками. | **Чтобы понять, сколько всего burst'ов** и взять время первого burst'а (`<azimuthTime>`). |
| **`<azimuthTime>`** | `annotation/...xml` -> `<swathTiming>` -> `<burstList>` -> `<burst>` | Время нулевого доплера для центра burst'а. | **Это `t_first_burst`** для расчета времени всех остальных burst'ов. |
| **`<linesPerBurst>`** | `annotation/...xml` -> `<swathTiming>` | Количество строк в одном burst'е (около 1500 для IW). | **Для расчета номера burst'а** по номеру строки (`N = floor(L / linesPerBurst) + 1`). |
| **`<azimuthTimeInterval>`** | `annotation/...xml` -> `<swathTiming>` | Время между двумя соседними строками в секундах. | **Для расчета времени burst'а.** `t_burst = t_first_burst + (N-1) * linesPerBurst * azimuthTimeInterval`. |
| **`<numberOfSamples>`** | `annotation/...xml` -> `<imageInformation>` | Количество отсчетов по дальности в каждой строке. | **Чтобы знать размерность данных** и создать вектор углов для каждого пикселя. |
| **`<slantRangeTime>`** | `annotation/...xml` -> `<imageInformation>` | Время запаздывания сигнала для первого пикселя по дальности. | **Для построения вектора времени по дальности `τ`.** |

---

### 3. Вспомогательные данные (Калибровка) — "Как это исправить?"

Это внешние файлы, которые содержат информацию для коррекции.

| Данные | Источник | Что это такое | Как мы это используем |
| :--- | :--- | :--- | :--- |
| **`AUX_CAL` файл** | Скачивается отдельно с сервера ESA (например, `S1A_AUX_CAL_V...SAFE`). | Файл калибровки. Содержит комплексные значения двухсторонней диаграммы направленности антенны (EAP) для каждой под-полосы и поляризации. | **Это ядро коррекции.** Мы извлекаем из него комплексный вектор `G_eap` и шаг угла `Δθ` для нужной под-полосы. |
| **`<elevationAngleIncrement>`** | `AUX_CAL` -> `<elevationAntennaPattern>` | Шаг угла между соседними значениями в векторе EAP (обычно 0.05°). | **Для построения вектора углов `θ_AM`** в системе координат антенны. |
| **`<values>`** | `AUX_CAL` -> `<elevationAntennaPattern>` | Комплексные числа (I, Q, I, Q...), представляющие диаграмму EAP. | **Это сам вектор `G_eap`**, который мы будем интерполировать. |

---

### 4. Обрабатывающая логика (Модели и формулы) — "По каким законам это работает?"

Это математические модели и алгоритмы, взятые из технической документации ESA.

| Данные/Модель | Источник | Что это такое | Как мы это используем |
| :--- | :--- | :--- | :--- |
| **Модель высоты орбиты** | `Sentinel-1-IPF_EAP_Phase_correction.pdf` (раздел 7.2) | Функция `s1_anx2Height(Δ_t)`. Преобразует время от ANX в высоту спутника. | **Шаг 1 для расчета угла крена.** По `Δ_t` получаем высоту `H`. |
| **Модель крена** | `Sentinel-1-IPF_EAP_Phase_correction.pdf` (раздел 7.1) | Функция `s1_height2roll(H)`. Преобразует высоту спутника в угол крена антенны. | **Шаг 2 для расчета угла крена.** По высоте `H` получаем `θ_roll`. |
| **`θ_roll`** | Результат вычислений | Угол наклона антенны в данный момент времени. | **Для преобразования углов:** `θ_view = θ_antenna + θ_roll`. |
| **`θ_AM`** | Рассчитывается из `AUX_CAL` | Вектор углов в системе координат антенны. | **Для преобразования углов:** `θ_view = θ_AM + θ_roll`. |
| **`θ_view`** | Результат вычислений | Вектор реальных углов обзора, соответствующий значениям EAP. | **Это ось X для интерполяции.** |
| **`θ_pixel`** | Рассчитывается из метаданных SLC | Вектор углов для каждого пикселя (столбца) в вашем burst'е. | **Это ось X для интерполяции.** Мы ищем, какие значения EAP соответствуют этим углам. |

---

### Как все это вместе работает (схема потока данных)

1.  **Фильтрация:** Берем список `.pkl` файлов. Используя метаданные из XML (номер строки, `linesPerBurst`, `t_first_burst`, `sliceList`), рассчитываем время каждого burst'а и отбираем только те, что относятся к slice #15.
2.  **Цикл по burst'ам:** Для каждого отфильтрованного burst'а:
    a. Берем его `azimuthTime`.
    b. Из `manifest.safe` берем `ascendingNodeTime`.
    c. Вычисляем `Δ_t`.
    d. Подставляем `Δ_t` в `s1_anx2Height` -> получаем `H`.
    e. Подставляем `H` в `s1_height2roll` -> получаем `θ_roll`.
    f. Из `AUX_CAL` берем `G_eap` и строим `θ_AM`.
    g. Вычисляем `θ_view = θ_AM + θ_roll`.
    h. Из метаданных SLC вычисляем `θ_pixel` для каждого столбца.
    i. Интерполируем `G_eap` от `θ_view` к `θ_pixel`.
    j. Делим наш 2D-массив burst'а на интерполированный вектор `G_eap`.
3.  **Результат:** Получаем скорректированный burst. Повторяем для всех.

Теперь у вас есть полная карта всех компонентов. Это должно помочь вам уверенно двигаться вперед.

# Алгоритмы SENTINEL-1 A ROLL STEERING LAW и их связь с данными калибровки

## Введение

Система SENTINEL-1 использует сложные алгоритмы управления креном (roll steering) для обеспечения точного направления антенны во время съемки. Эти алгоритмы тесно связаны с данными калибровки антенны, особенно с двухсторонним шаблоном диаграммы направленности по углу места (Two-way Elevation Antenna Pattern, EAP). Рассмотрим подробнее работу этих алгоритмов и их взаимосвязь с данными калибровки.

## 1. Алгоритмы управления креном (Roll Steering Law)

В документе Sentinel-1-IPF_EAP_Phase_correction.pdf описаны два взаимосвязанных алгоритма управления креном антенны:

### 1.1. Roll Steering Law vs. высота (раздел 7.1)

Этот алгоритм определяет угол отклонения механической оси антенны от надира в зависимости от высоты спутника над поверхностью Земли.

**Основная формула:**
```
θroll_steering = θref - αroll * (H - Href)
```

**Параметры:**
- `θref` = 29.450° - угол отклонения антенны от надира при опорной высоте
- `Href` = 711.700 км - опорная высота
- `αroll` = 0.05660°/км - коэффициент чувствительности управления креном к изменению высоты
- `H` - текущая высота спутника в километрах

**Реализация функции `s1_height2roll`:**
```python
function s1_height2roll, altitude
    href = 711.700  # км
    boresight_ref = 29.450  # градусов
    alpha_roll = 0.0566  # градусов/км
    
    return, boresight_ref - alpha_roll * (altitude/1000.0 - href)
end
```

Здесь высота спутника `altitude` подается в метрах, поэтому для преобразования в километры выполняется деление на 1000.

### 1.2. Roll Steering Law vs. время (раздел 7.2)

Этот алгоритм позволяет вычислить высоту спутника как функцию времени с момента пересечения восходящего узла орбиты (ANX), а затем использовать эту высоту для вычисления угла управления креном.

**Формула высоты спутника:**
```
H(t) = h0 + Σ[hi * sin(i * ωorb * t + φi)], где i = 1..4
```

**Параметры:**
- `h0` = 707714.8 м - базовая высота
- `h1` = 8351.5 м, `h2` = 8947.0 м, `h3` = 23.32 м, `h4` = 11.74 м - амплитудные коэффициенты
- `φ1` = 3.1495 рад, `φ2` = -1.5655 рад, `φ3` = -3.1297 рад, `φ4` = 4.7222 рад - фазовые коэффициенты
- `Torb` = 5924.57 с - период орбиты (175 орбит за 12 дней)
- `ωorb` = 2π/Torb - угловая частота орбитального движения

**Реализация функции `s1_anx2Height`:**
```python
function s1_anx2Height, anxTime
    h0 = 707714.8  # м
    h = [8351.5, 8947.0, 23.32, 11.74]  # м
    phi = [3.1495, -1.5655, -3.1297, 4.7222]  # радианы
    Torb = (12 * 24 * 60 * 60) / 175  # секунды
    worb = 2 * π / Torb
    
    sum = 0
    for i = 0, n_elements(h)-1 do begin
        sum += h(i) * sin((i+1) * worb * anxTime + phi[i])
    endfor
    
    Height = h0 + sum
    return, Height
end
```

**Комбинированная функция `s1_anx2roll`:**
Эта функция сначала вычисляет высоту спутника для заданного времени с момента ANX с помощью `s1_anx2Height`, а затем использует эту высоту для вычисления угла управления креном с помощью `s1_height2roll`.

### 1.3. Процесс вычисления угла управления креном в полной процедуре

В разделе 5.3 документа описан полный процесс вычисления угла управления креном:

1. Вычисление времени, прошедшего с момента пересечения восходящего узла (ANX):
   ```
   Δt = ηburst_start - ηanx
   ```
   где:
   - `ηburst_start` - время начала пачки (burst) или блока данных
   - `ηanx` - время пересечения восходящего узла орбиты

2. Вычисление высоты спутника для этого времени с помощью функции `s1_anx2Height`

3. Вычисление угла управления креном с помощью функции `s1_height2roll`:
   ```
   θroll_steering = anx2roll(Δt)
   ```

## 2. Связь с данными калибровки AUX_CAL (раздел 6)

Данные из файла калибровки AUX_CAL критически важны для корректного применения поправок на фазу EAP. Давайте рассмотрим, какие именно данные используются и как они связаны с алгоритмами управления креном.

### 2.1. Что содержится в файле AUX_CAL

Согласно разделу 6 документа Sentinel-1-IPF_EAP_Phase_correction.pdf и разделу 5 "CALIBRATION AUXILIARY DATA" спецификации S1-RS-MDA-52-7443, файлы AUX_CAL содержат следующие ключевые параметры для каждого поддиапазона (sub-swath) и поляризации:

1. **beamNominalNearRange** (26.72° в примере) - угол места номинальной ближней границы луча. Важно отметить, что согласно спецификации, этот параметр на практике не используется в IPF.

2. **beamNominalFarRange** (31.67° в примере) - угол места номинальной дальней границы луча. Также не используется в IPF.

3. **elevationAngleIncrement** (0.05° в примере) - шаг угла места между последовательными значениями в шаблоне диаграммы направленности. Это критически важный параметр, используемый для построения вектора углов места в системе координат антенны (θAM).

4. **values** - массив комплексных значений (I,Q), представляющих двухсторонний шаблон антенны по углу места. Число элементов в этом массиве всегда нечетное (Nelt).

### 2.2. Как данные AUX_CAL используются в процедуре коррекции

Согласно разделу 5.2 документа, данные из AUX_CAL используются в следующем порядке:

1. **Извлечение комплексного EAP**:
   - Для нужного поддиапазона (sub-swath) и поляризации извлекается комплексный вектор Geap
   - Этот вектор содержит Nelt комплексных значений, где Nelt - нечетное число

2. **Вычисление вектора углов места в системе координат антенны**:
   ```
   θAM = [-(Nelt-1)/2, ..., (Nelt-1)/2] * Δθ
   ```
   где Δθ - это elevationAngleIncrement из AUX_CAL (0.05° в примере)

3. **Преобразование в геометрию наблюдения**:
   С помощью угла управления креном (вычисленного по алгоритмам раздела 7) вектор углов места преобразуется в углы места в геометрии наблюдения:
   ```
   θgeom = θAM + θroll_steering
   ```

4. **Интерполяция комплексного EAP**:
   - Комплексные значения EAP интерполируются для получения значений для каждого отсчета по дальности в продукте
   - Интерполяция выполняется в комплексной области для избежания разрывов фазы

5. **Применение фазовой поправки**:
   Для продуктов, обработанных с IPF версии 236 (без коррекции фазы EAP), применяется поправка:
   ```
   scorr(η, τ) = s(η, τ) / exp(j * Φeap_interpolated(τ))
   ```
   где:
   - s(η, τ) - исходные данные SLC, обработанные IPF V236
   - Φeap_interpolated(τ) - интерполированная фаза EAP для каждого отсчета по дальности τ

### 2.3. Критическая роль угла управления креном

Угол управления креном служит связующим звеном между системой координат антенны (где определен шаблон EAP) и геометрией наблюдения (где находится цель):

- Шаблон антенны по углу места (EAP) определен в системе координат антенны относительно опорного угла (referenceAntennaAngle)
- При съемке антенна направлена под углом управления креном относительно надира
- Для корректного применения шаблона к данным необходимо преобразовать углы из системы координат антенны в геометрию наблюдения
- Это преобразование выполняется именно путем добавления угла управления креном к углам места в системе координат антенны

Таким образом, даже небольшая ошибка в вычислении угла управления креном приведет к ошибке в применении фазовой поправки EAP, что особенно критично при создании интерферограмм между продуктами, обработанными разными версиями IPF.

## Заключение

Алгоритмы управления креном SENTINEL-1 (Roll Steering Law) представляют собой сложную, но математически строгую систему, которая обеспечивает точное направление антенны спутника. Эти алгоритмы тесно связаны с данными калибровки антенны, особенно с двухсторонним шаблоном диаграммы направленности по углу места (EAP).

В процедуре коррекции интерферограмм:
1. Из AUX_CAL извлекаются ключевые параметры: elevationAngleIncrement и комплексные значения EAP
2. На основе времени начала пачки и времени восходящего узла вычисляется угол управления креном с использованием алгоритмов раздела 7
3. Углы места преобразуются из системы координат антенны в геометрию наблюдения с помощью полученного угла управления креном
4. Выполняется интерполяция комплексного EAP и применяется фазовая поправка к данным

Эта процедура обеспечивает совместимость между продуктами, обработанными разными версиями IPF, и позволяет сохранять интерферометрическую фазу при создании временных рядов Sentinel-1.

# Какие данные необходимо извлечь из файлов для алгоритмов управления креном и калибровки

Для вычислений в алгоритмах управления креном (roll steering law) и коррекции фазы EAP (Elevation Antenna Pattern) из предоставленных файлов необходимо извлечь следующие данные:

## 1. Из файла manifest_slc.txt:

### Критически важные данные:
- **Время восходящего узла орбиты (ANX time)**:
  ```xml
  <s1:ascendingNodeTime>2025-10-25T17:11:18.931874</s1:ascendingNodeTime>
  ```
  Это время пересечения спутником экватора при движении с юга на север, необходимое для определения времени, прошедшего с ANX для вычисления угла управления креном.

- **Версия IPF (Instrument Processing Facility)**:
  ```xml
  <safe:software name="Sentinel-1 IPF" version="004.00"/>
  ```
  Версия процессора определяет, требуется ли применение фазовой поправки EAP. Для IPF версии 2.36 (V236) поправка требуется, для версий 2.43 (V243) и выше - нет.

- **Время начала и окончания съемки**:
  ```xml
  <safe:startTime>2025-10-25T17:25:19.391523</safe:startTime>
  <safe:stopTime>2025-10-25T17:25:46.471422</safe:stopTime>
  ```

- **Время с начала восходящего узла (time ANX)**:
  ```xml
  <s1:timeANX>
    <s1:startTimeANX>8.404596e+05</s1:startTimeANX>
    <s1:stopTimeANX>8.675396e+05</s1:stopTimeANX>
  </s1:timeANX>
  ```
  Эти значения выражены в микросекундах с момента ANX и используются для точного определения времени начала пачек.

### Ссылки на вспомогательные файлы:
```xml
<safe:resource name="/data/CDP/production//2927960/S1A_AUX_CAL_V20190228T092500_G20240327T102320.SAFE" role="AUX_CAL"/>
```
Эта ссылка указывает на файл калибровки AUX_CAL, содержащий комплексные значения EAP, критически важные для фазовой поправки.

## 2. Из файла s1a-iw1-slc-vv-2025....txt (аннотация продукта):

### Для управления креном:
- **Времена начала пачек (burst start times)**:
  ```xml
  <swathTiming>
    <burstList>
      <burst>
        <azimuthTime>...</azimuthTime>
        ...
      </burst>
      ...
    </burstList>
  </swathTiming>
  ```
  Для каждой пачки необходимо извлечь время начала в формате UTC для вычисления времени, прошедшего с момента ANX.

### Для калибровки EAP:
- **Данные антенного шаблона по углу места**:
  ```xml
  <antennaPattern>
    <antennaPatternList>
      <antennaPattern>
        <azimuthTime>...</azimuthTime>
        <slantRangeTime>...</slantRangeTime>
        <elevationAngle>...</elevationAngle>
        <elevationPattern>...</elevationPattern>
        <incidenceAngle>...</incidenceAngle>
        <terrainHeight>...</terrainHeight>
        <roll>...</roll>
      </antennaPattern>
      ...
    </antennaPatternList>
  </antennaPattern>
  ```
  Важны значения `elevationAngle` и `elevationPattern` для преобразования углов места. Значения `roll` также могут быть полезны для проверки.

- **Скорость дискретизации по дальности**:
  ```xml
  <rangeSamplingRate>...</rangeSamplingRate>
  ```

- **Количество выборок**:
  ```xml
  <numberOfSamples>...</numberOfSamples>
  <numberOfLines>...</numberOfLines>
  ```

- **Slant range time начала изображения**:
  ```xml
  <slantRangeTime>...</slantRangeTime>
  ```

## 3. Данные, которые необходимо извлечь из AUX_CAL (указан в manifest):

Хотя файл AUX_CAL не предоставлен, из спецификации S1-RS-MDA-52-7443 известно, что необходимо извлечь:

- **Комплексные значения двухстороннего шаблона антенны по углу места (EAP)**:
  ```xml
  <calibrationParams>
    <swath>IW1</swath>
    <polarisation>VV</polarisation>
    <elevationAntennaPattern>
      <values count="601">...</values>
    </elevationAntennaPattern>
  </calibrationParams>
  ```

- **Шаг угла места (elevation angle increment)**:
  ```xml
  <elevationAngleIncrement>0.05</elevationAngleIncrement>
  ```

- **Абсолютную калибровочную константу** (для полной калибровки):
  ```xml
  <absoluteCalibrationConstant>1.0</absoluteCalibrationConstant>
  ```

## Процедура использования данных:

1. **Для вычисления угла управления креном**:
   - Вычислить разницу во времени между временем начала пачки и временем ANX
   - Использовать эту разницу для вычисления высоты спутника с помощью функции `s1_anx2Height`
   - Вычислить угол управления креном с помощью функции `s1_height2roll`, используя высоту спутника

2. **Для коррекции фазы EAP**:
   - Проверить версию IPF из manifest
   - Если версия < 243, извлечь комплексные значения EAP из AUX_CAL для соответствующего поддиапазона (swath) и поляризации
   - Вычислить вектор углов места в системе координат антенны
   - Преобразовать углы места в геометрию наблюдения с использованием угла управления креном
   - Интерполировать комплексные значения EAP для каждого отсчета по дальности
   - Применить фазовую поправку к исходным данным SLC

Таким образом, из предоставленных файлов необходимо извлечь временные параметры съемки, параметры антенного шаблона и информацию о версии обработки, а также получить доступ к файлу AUX_CAL для комплексных значений EAP.